# Sprint13課題 深層学習スクラッチ畳み込みニューラルネットワーク2

## 【問題1】2次元畳み込み層の作成

フォワードプロパゲーションの数式
$$
a_{i,j,m} = \sum_{k=0}^{K-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1}x_{(i+s),(j+t),k}w_{s,t,k,m}+b_{m}
$$

更新式
$$
w_{s,t,k,m}^{\prime} = w_{s,t,k,m} - \alpha \frac{\partial L}{\partial w_{s,t,k,m}} \\
b_{m}^{\prime} = b_{m} - \alpha \frac{\partial L}{\partial b_{m}}
$$

バックプロパゲーションの数式
$$
\frac{\partial L}{\partial w_{s,t,k,m}} = \sum_{i=0}^{N_{out,h}-1}\sum_{j=0}^{N_{out,w}-1} \frac{\partial L}{\partial a_{i,j,m}}x_{(i+s)(j+t),k}\\
\frac{\partial L}{\partial b_{m}} = \sum_{i=0}^{N_{out,h}-1}\sum_{j=0}^{N_{out,w}-1}\frac{\partial L}{\partial a_{i,j,m}}
$$

前の層に流す誤差の数式
$$
\frac{\partial L}{\partial x_{i,j,k}} = \sum_{m=0}^{M-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1} \frac{\partial L}{\partial a_{(i-s),(j-t),m}}w_{s,t,k,m}
$$

[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)のpyファイルにConv2Dクラスを実装

## 【問題2】2次元畳み込み後の出力サイズ
$$
N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1\\
N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1
$$

[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/utils/change_shape.py)のget_output_sizeで実装<br>
[Conv2DやMaxPooling](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)などのクラスの中で使用

## 【問題3】最大プーリング層の作成
$$
a_{i,j,k} = \max_{(p,q)\in P_{i,j}}x_{p,q,k}
$$

[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)のpyファイルにMaxPoolingクラスを実装

## 【問題4】平滑化
平滑化するためのクラスFlatten()を作成してください。

フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。

[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)のpyファイルにFlattenクラスを実装

## 【問題5】学習・推定
作成したConv2dを使用してMNISTの分類を学習・推定してください。

この段階では精度は気にせず、動くことを確認してください。

[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/scratch_cnn2d_classifier.py)のpyファイルにScratch2dCNNClassifierクラスを実装

In [2]:
#データセットの用意
from keras.datasets import mnist
import numpy as np
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#reshape
X_train = X_train.reshape(len(X_train), 1, 28, 28)
X_test = X_test.reshape(len(X_test), 1, 28, 28)
#one-hot
eye = np.eye(len(np.unique(y_train)))
y_train = eye[y_train]
y_test = eye[y_test]

#normalize
X_train = X_train.astype(np.float)/255
X_test = X_test.astype(np.float)/255
X_train.shape

#split into train, val
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print(X_train.shape) 
print(X_val.shape) 

Using TensorFlow backend.


(48000, 1, 28, 28)
(12000, 1, 28, 28)


In [3]:
import sys

dir_str = "../../ml-scratch/model"
if (dir_str not in sys.path):
    sys.path.append(dir_str)

from scratch_cnn2d_classifier import Scratch2dCNNClassifier
cnn = Scratch2dCNNClassifier(
    conv_param={'n_filters': 30, 'filter_size': 5, 'stride': 1, 'pad': 0},
    pool_param={'pool_size': 2},
    n_epochs=3,
    batch_size=1000,
    optimizer='Adam',
    optimizer_param={'lr': 0.001},
    layer_nodes = {'hidden': 100, 'output': 10},
    weight_init_std=0.01,
    verbose=True
)
cnn.fit(x_train=X_train, y_train=y_train, x_val=X_val, y_val=y_val)
pred = cnn.predict(X_test)

epoch: 0
train_acc: 0.8714375, val_acc0.8705
train loss: 0.43093932640887195, val_loss0.43093932640887195
epoch: 1
train_acc: 0.9168958333333334, val_acc0.9154166666666667
train loss: 0.2781913655388082, val_loss0.2781913655388082
epoch: 2
train_acc: 0.9363958333333333, val_acc0.935
train loss: 0.2119372382259603, val_loss0.2119372382259603


In [4]:
y_actual = np.argmax(y_test, axis=1)
print("pred: ", pred)
print("actual: ", y_actual)
print("acuracy: ", (pred == y_actual).sum() / len(pred))

pred:  [7 2 1 ... 4 5 6]
actual:  [7 2 1 ... 4 5 6]
acuracy:  0.9405


## 【問題6】（アドバンス課題）LeNet
CNNで画像認識を行う際は、フィルタサイズや層の数などを１から考えるのではなく、有名な構造を利用することが一般的です。

現在では実用的に使われることはありませんが、歴史的に重要なのは1998年のLeNetです。この構造を再現して動かしてみましょう。

## 【問題7】（アドバンス課題）有名な画像認識モデルの調査
CNNの代表的な構造としてははAlexNet(2012)、VGG16(2014)などがあります。こういったものはフレームワークで既に用意されていることも多いです。

どういったものがあるか簡単に調べてまとめてください。名前だけでも見ておくと良いでしょう。

参考

Applications - Keras Documentation

## 【問題8】（アドバンス課題）平均プーリングの作成
平均プーリング層のクラスAveragePool2Dを作成してください。

範囲内の最大値ではなく、平均値を出力とするプーリング層です。

画像認識関係では最大プーリング層が一般的で、平均プーリングはあまり使われません。

## 【問題9】出力サイズとパラメータ数の計算
CNNモデルを構築する際には、全結合層に入力する段階で特徴量がいくつになっているかを事前に計算する必要があります。

また、巨大なモデルを扱うようになると、メモリや計算速度の関係でパラメータ数の計算は必須になってきます。フレームワークでは各層のパラメータ数を表示させることが可能ですが、意味を理解していなくては適切な調整が行えません。

以下の3つの畳み込み層の出力サイズとパラメータ数を計算してください。パラメータ数についてはバイアス項も考えてください。
＊最後の例は丁度良く畳み込みをすることができない場合です。フレームワークでは余ったピクセルを見ないという処理が行われることがあるので、その場合を考えて計算してください。端が欠けてしまうので、こういった設定は好ましくないという例です。

1.<br>

入力サイズ : 144×144, 3チャンネル<br>
フィルタサイズ : 3×3, 6チャンネル<br>
ストライド : 1<br>
パディング : なし<br>
<br>
畳み込み層の出力サイズ: <br>
1 +(144 + 2*0 - 3)/1 = 142より142^2のサイズで6 channel<br>
パラメータ数: <br>
filter_size * num_input_channels * num_filters = (3 * 3) * 3 * (6/3) = 54<br>
バイアス項をフィルター枚数分足すと56<br>
<br>


2.<br>
入力サイズ : 60×60, 24チャンネル<br>
フィルタサイズ : 3×3, 48チャンネル<br>
ストライド　: 1<br>
パディング : なし<br>
<br>
畳み込み層の出力サイズ: <br>
1 +(60 + 2*0 - 3)/1 = 58より58^2のサイズで48 channel<br>
パラメータ数: <br>
filter_size * num_input_channels * num_filters = (3 * 3) * 24 * (48/24) = 432<br>
バイアス項をフィルター枚数分足すと434<br>
<br>
3.<br>
入力サイズ : 20×20, 10チャンネル<br>
フィルタサイズ: 3×3, 20チャンネル<br>
ストライド : 2<br>
パディング : なし<br>
<br>
畳み込み層の出力サイズ: <br>
1 +(20 + 2*0 - 3)/2 = 9.5 より9^2のサイズで20 channel<br>
パラメータ数: <br>
filter_size * num_input_channels * num_filters =  (3 * 3) * 10 * (20/10) = 180<br>
バイアス項をフィルター枚数分足すと182<br>
<br>

## 【問題10】（アドバンス課題）フィルタサイズに関する調査
畳み込み層にはフィルタサイズというハイパーパラメータがありますが、2次元畳み込み層において現在では3×3と1×1の使用が大半です。以下のそれぞれを調べたり、自分なりに考えて説明してください。

7×7などの大きめのものではなく、3×3のフィルタが一般的に使われる理由
高さや幅方向を持たない1×1のフィルタの効果

H_out = (H_in + 2P - F)/S + 1
に関して、ストライドが1の場合
H_out = H_in + 2P - F + 1
となる。
2P - F + 1が偶数の時、H_outとH_inでの画像の偶奇が不変となる。
この時、2Pは偶数なので、-F+1が偶数（=Fが奇数）である。

フィルターサイズが奇数である理由は上記のようなものだと考えられる。
ストライドが1であるのは除算で割り切れない場合を避けているのだろうか。